In [13]:
import time
import astroquery.heasarc
from astropy.coordinates import SkyCoord
import astropy.units as u

from astroquery.jplhorizons import Horizons
import numpy as np
import pandas as pd

from collections import defaultdict

from astropy.time import Time

import getpass
import oda_api.token

import logging

from oda_api.api import DispatcherAPI

from oda_api.plot_tools import OdaImage

import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from matplotlib.patches import Circle

Idea of how this code works (27/02)

1. We obtain Jupiter's position in the sky (ephemerides). We will be using a search radius of 8 degrees (due to FOV of INTEGRAL, fully coded). I found that Jupiter's position does not significantly change (more than 8 degrees) within less than 2 months. For safety, we will obtain Jupiter's mean monthly position.
2. Using the mean monthly position of Jupiter over a certain time period (1 year, 2 years, ...), we will search for all the ScWs within 8 degrees of this position. No time filtering yet. We will group by mean monthly position of Jupiter.
Though we already filter for pointing ScWs only.
3. Then, we have N groups of ScWs, each corresponding to a certain RA and DEC (within 8 degrees). These only contain Jupiter if it is in the FOV at that precise time, i.e. that month. So filtering will be done by taking ScWs only within corresponding month (aka the month corresponding to the mean position as found in step 1).
4. Finally, we will have N groups of ScWs, each containg Jupiter, for each month in the given interval. Some may be empty.

STEP 1 - obtaining Jupiter's position

In [14]:
# First, define the time interval we are interested in (> 1 month)
epochs = {'start':'2003-01-01', 'stop':'2004-01-01', 'step':'31d'}

# Get Jupiter's monthly position in the given time interval (not using mean as step of 31 days)
jupiter = Horizons(id='599', location='@0',epochs=epochs) 
eph = jupiter.ephemerides()

ra_values = eph['RA']
dec_values = eph['DEC']
print(f"Jupiter's monthly position between {epochs['start']} and {epochs['stop']}: \n")
print(eph[['datetime_str', 'RA', 'DEC']])

# If we want to use mean:
# df = eph.to_pandas()
# df['datetime_str'] = pd.to_datetime(df['datetime_str'])
# monthly_means = df.groupby(df['datetime_str'].dt.strftime('%Y-%m'))[['RA', 'DEC']].mean()
# for month, row in monthly_means.iterrows():
    # print(f"Month: {month}, Mean RA: {row['RA']:.2f} deg, Mean DEC: {row['DEC']:.2f} deg")

Jupiter's monthly position between 2003-01-01 and 2004-01-01: 

   datetime_str       RA      DEC   
       ---           deg      deg   
----------------- --------- --------
2003-Jan-01 00:00 133.13461 18.23402
2003-Feb-01 00:00 135.64353 17.59249
2003-Mar-04 00:00  138.1248 16.92216
2003-Apr-04 00:00 140.57899 16.22478
2003-May-05 00:00 143.00678 15.50206
2003-Jun-05 00:00 145.40891  14.7557
2003-Jul-06 00:00 147.78632 13.98736
2003-Aug-06 00:00    150.14 13.19866
2003-Sep-06 00:00 152.47101 12.39117
2003-Oct-07 00:00 154.78053 11.56645
2003-Nov-07 00:00 157.06978 10.72599
2003-Dec-08 00:00 159.34001  9.87127


STEP 2 - obtaining ScWs

In [15]:
Heasarc = astroquery.heasarc.Heasarc()

def get_scw_list(ra_obj, dec_obj, radius):
    R = Heasarc.query_region(
            position = SkyCoord(ra_obj, dec_obj, unit='deg'),
            radius = f"{radius} deg",
            catalog = 'intscw',
            good_isgri = ">1000",
        )
    R.sort('scw_id') 
    return R['scw_id'], R['scw_ver'], R['start_date'], R['end_date']

assert astroquery.__version__ >= '0.4.2.dev6611'
assert 'isdc' in astroquery.heasarc.Conf.server.cfgtype

In [ ]:
scw_ids = []
scw_versions = []
scw_start_times = []
scw_end_times = []

for row in eph:
    ra = float(row['RA'])
    dec = float(row['DEC'])
    radius = 8.
    scw_id, scw_ver, scw_start, scw_end = get_scw_list(ra, dec, radius)
    scw_ids.append(scw_id) # Using append means we get a list of lists, grouped by Jupiter's monthly position
    scw_versions.append(scw_ver)
    scw_start_times.append(scw_start)
    scw_end_times.append(scw_end)

# Filter by type (pointing type)
scw_ids_filtered = []
scw_versions_filtered = []
scw_start_times_filtered = []
scw_end_times_filtered = []

for scw_sublist, ver_sublist, start_sublist, end_sublist in zip(scw_ids, scw_versions, scw_start_times, scw_end_times):
    
    mask = [int(scw) % 10 == 0 for scw in scw_sublist]
    
    filtered_scws = [scw_sublist[i] for i in range(len(scw_sublist)) if mask[i]]
    filtered_versions = [ver_sublist[i] for i in range(len(ver_sublist)) if mask[i]]
    filtered_start_times = [start_sublist[i] for i in range(len(start_sublist)) if mask[i]]
    filtered_end_times = [end_sublist[i] for i in range(len(end_sublist)) if mask[i]]
    
    scw_ids_filtered.append(filtered_scws)
    scw_versions_filtered.append(filtered_versions)
    scw_start_times_filtered.append(filtered_start_times)
    scw_end_times_filtered.append(filtered_end_times)

scw_ids = scw_ids_filtered
scw_versions = scw_versions_filtered
scw_start_times = scw_start_times_filtered
scw_end_times = scw_end_times_filtered

print("Found a total of", np.sum([len(scw_id) for scw_id in scw_ids]), "ScWs.")
print("Number of ScWs found per month: \n")
for i, scw_sublist in enumerate(scw_ids):
    print(f"Month {i+1} - # of ScWs found: {len(scw_sublist)}")


Found  12  pointing science windows


STEP 3 - filtering by time

In [23]:
scw_ids_filtered = []
scw_versions_filtered = []
scw_start_times_filtered = []
scw_end_times_filtered = []

for scw_sublist, ver_sublist, start_sublist, end_sublist in zip(scw_ids, scw_versions, scw_start_times, scw_end_times):
    
    mask = [
        (start >= Time(epochs['start'], format='isot').mjd) and
        (end <= Time(epochs['stop'], format='isot').mjd)
        for start, end in zip(start_sublist, end_sublist)
    ]    
    filtered_scws = [scw_sublist[i] for i in range(len(scw_sublist)) if mask[i]]
    filtered_versions = [ver_sublist[i] for i in range(len(ver_sublist)) if mask[i]]
    filtered_start_times = [start_sublist[i] for i in range(len(start_sublist)) if mask[i]]
    filtered_end_times = [end_sublist[i] for i in range(len(end_sublist)) if mask[i]]
    
    scw_ids_filtered.append(filtered_scws)
    scw_versions_filtered.append(filtered_versions)
    scw_start_times_filtered.append(filtered_start_times)
    scw_end_times_filtered.append(filtered_end_times)

scw_ids = scw_ids_filtered
scw_versions = scw_versions_filtered
scw_start_times = scw_start_times_filtered
scw_end_times = scw_end_times_filtered

print("Found a total of", np.sum([len(scw_id) for scw_id in scw_ids]), "matching ScWs.")
print("Number of matching ScWs found per month: \n")
for i, scw_sublist in enumerate(scw_ids):
    print(f"Month {i+1} - # of ScWs found: {len(scw_sublist)}")    

Found a total of 71 matching ScWs.
Number of matching ScWs found per month: 

Month 1 - # of ScWs found: 6
Month 2 - # of ScWs found: 6
Month 3 - # of ScWs found: 12
Month 4 - # of ScWs found: 9
Month 5 - # of ScWs found: 11
Month 6 - # of ScWs found: 9
Month 7 - # of ScWs found: 9
Month 8 - # of ScWs found: 9
Month 9 - # of ScWs found: 0
Month 10 - # of ScWs found: 0
Month 11 - # of ScWs found: 0
Month 12 - # of ScWs found: 0
